<a href="https://colab.research.google.com/github/ol3gka/AI_Masrters_NLA_project_1_Matrix_Masters/blob/main/Maxvol_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# maxvol-2 (rect-maxvol)
Автор: Николаев О.В.




In [ ]:
# Ensure: A submatrix Ab, a set of pivot rows {p} and a matrix of coefficients C
# such, that A = CA, b ∀i /∈ {p} : kCik2 ≤ τ


# 1: Start with a non-singular square submatrix Ab {Result of the maxvol}
# 2: {p} ← pivot rows; C ← AAb−1
# ; ∀i : Li ← kCik
# 2
# 2
# {Result of the maxvol}
# 3: i ← argmaxi/∈{p}
# (Li) {Find maximal row in C}
# 4: while Li > τ 2 do
# 5: {p} ← {p} + i {Extend set of pivots}
# 6: Ab ←
# 
# Ab
# Ai
# 
# {Extend Ab}
# 7: C ←
# h
# C −
# CC∗
# i Ci
# 1+CiC∗
# i
# CC∗
# i
# 1+CiC∗
# i
# i
# {Rank-1 update of C}
# 8: ∀j : Lj ← Lj −
# |CjC
# ∗
# i
# |
# 2
# 1+CiC∗
# i
# {Update lengths of rows of C}
# 9: i ← argmaxi/∈{p}
# (Li) {Find maximal row in C}
# 10: end while
# 11: if Cb is required to be identity then
# 12: Cb = I
# 13: end if
# 14: return C, A, b {p}

**Algorithm 1 rect_maxvol (“Greedy” maximization of the volume of submatrix)**

$
\begin{array}{l}
\text {Require: A full-rank A} \\

N > r, parameter τ

\end{array}
$

$
\begin{array}{l}
\text { procedure PUSHCONDLOCAL }\left(r_{p r e}, r_{c u r}, \text { phase }\right) \\
\text { if !PUSHCOND }\left(r_{p r e}, \text { phase }\right) \text { then } \\
\text { if PUSHCOND }\left(r_{c u r}, \text { phase }\right) \text { then } \\
\text { return true } \\
\text { return } f \text { alse } \\
\text { procedure OPTPARALLELPUSH }\left(P_{s}, R_{s}, F Q, p h a s e\right) \\
F Q^{\prime}=\emptyset \\
E=\emptyset \\
\text { parallel for } u \in F Q \\
r_{u} \leftarrow R_{s}(u) \\
E=E \cup\left(u, r_{u}\right) \\
\text { parallel for } v \in N_{i n}(u) \\
\quad i n c=(1-\alpha) \cdot r_{u} / d_{o u t}(v) \\
\quad r_{p r e}=\operatorname{ATOMICADD}\left(R_{s}(v), \text { inc }\right) \\
\quad r_{c u r}=r_{p r e}+i n c \\
\quad \text { if PUSHCONDLOCAL }\left(r_{p r e}, r_{c u r}, p h a s e\right) \text { then } \\
\quad \text { ENQUEUE }\left(F Q^{\prime}, v\right) \\
\text { synchronize } \\
\text { parallel for }\left(u, r_{u}\right) \in E \\
P_{s}(u)+=\alpha \cdot r_{u} \\
R_{s}(u)-=r_{u} \\
\quad \text { if } \operatorname{PUSHCOND}\left(R_{s}(u), p h a s e\right) \text { then } \\
\quad \text { ENQUEUE }\left(F Q^{\prime}, u\right) \\
\text { synchronize } \\
\text { return } F Q^{\prime}
\end{array}
$